| title               | tags      | created | modified |
|---------------------|-----------|---------|----------|
| dev-rag-deepdoc-pdf | [dev,rag] |  Sun Mar 16 03:09:57 CST 2025 | Sun Mar 16 03:09:57 CST 2025  |

## 需求：
> 基于ragflow进行二次开发，对pdf文档添加图片解析

# 图片处理与语义增强

## 1. 图片提取

### v1: 使用 `pdfplumber` 提取图片
- **问题**: 提取的图片色域为 CMYK，保存后不是 RGB 通道。
- **尝试**: 使用 Pillow 的 `convert('RGB')` 转换，但效果无效。
- **可能原因**: 需要加载 ICC 或 ICM 文件进行色域转换。

### v2: 改用 `fitz` 提取图片流
- **流程**:
  1. 使用 `pdfplumber` 提取图片 ID。
  2. 将图片 ID 传递给 `fitz`，提取图片流。
- **优势**: 解决了色域转换问题，提高提取效率。

## 2. 图片语义增强
- **背景**: 将图片 URL 嵌入文本中，常规模型在召回时无法理解图片语义。
- **挑战**: 不使用 VL（视觉-语言）模型时，语义理解受限。
- **解决方案**: 使用 VL 模型进行图片增强。
- **当前选择**: 采用 `qwen-vl-max` 模型，提升图片语义理解能力。

### 2.1  图片增强嵌入
- **方案**: a. 图片提取时嵌入 b. 向量化分块前嵌入